In [ ]:
import torch, numpy as np, torch_geometric as pyg, torch.nn as nn, torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch_geometric.loader import DataLoader
#from torch_geometric.data import Dataset
from torch_geometric.data import HeteroData

from sklearn.model_selection import train_test_split

import g_model as gm
#from utils.nntool import SimpTrainer

import pickle as pckl

from tqdm._tqdm_notebook import tqdm

In [ ]:
data: list[ tuple[gm.gProblem_T, torch.Tensor] ]

with open(r"./gdata/", "rb") as f:
    data = pckl.load(f)
print(data)

maybe we can add some real data, and set sth 'oversampling rate' here ? 

In [ ]:
data: list[HeteroData] = []
for x_, y_ in data:
    hd_g = gm.build_ggraph(x_); hd_g["token"].y = y_
    data.append(hd_g)

print(data)

In [ ]:
loss_fn = nn.MSELoss()

In [ ]:
# fuck! i just can't use SimpTrainer here!

def get_loss(mdl: gm.TokenSemantics, dt: HeteroData) -> torch.Tensor:
    return loss_fn(mdl(dt), dt["token"].y)

In [ ]:
def train_epoch(mdl, loader: DataLoader, opt: optim.Optimizer):
    mdl.train()
    los_ = []
    for batch in tqdm(loader, leave = False, ):
        opt.zero_grad()
        ( loss := get_loss(mdl, batch) ).backward()
        opt.step()
        los_.append(loss.item())
    return np.mean(los_)

In [ ]:
tr, ts = train_test_split(data, test_size = 0.3)

tr = DataLoader(tr, batch_size = 5, shuffle = True)
for ts in DataLoader(ts): ...       # this is so confusing

In [ ]:
mdl = gm.TokenSemantics(64, 128)
opt = optim.AdamW(
    mdl.parameters(),
    lr = 1e-3,
    weight_decay = 1.5e-2,
)
sched = CosineAnnealingLR(
    opt,
    T_max = (EPOCH := 50),
    eta_min = 3e-5,
)

In [ ]:
for i in range(EPOCH):
    ls_tr = train_epoch(mdl, tr, opt)
    sched.step()
    
    print(f"(-{i})  ls_tr:{ls_tr:.4f}")
    if i % 10 == 9:
        mdl.eval()
        with torch.no_grad(): ls_ts = get_loss(mdl, ts)
        print(f"()  ls_ts:{ls_ts:.4f}")